# Session 12(a) - Text Mining
## Analysing and summarising collections of text
### Phrases

Having learned how to clean and simplify our text for processing, the next stage is to ask what our text is about. This session will cover finding

In [ ]:
import pandas as pd
import spacy

In [ ]:
df = pd.read_csv('sample_news_large.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['query'].value_counts()

In [ ]:
nlp = spacy.load('en_core_web_md')

## Phrases (n-grams)
<img src="https://github.com/Minyall/sc207_materials/blob/master/images/Archer-phrasing.jpg?raw=true" align="right" width="300">

- Operates on the assumption that if words often co-occur together in a corpus, they should be considered as a single 'phrase', rather than as individual words.
- Phrasing improves the accuracy of various analyses as it recognises that words may be transformed by their context.
- For example: 
    - In one document we have the phrase "human rights", in the other, "human biology". 
    - **Without phrasing** these may be considered similar as they both use the word "human".
    - However **with phrasing** these would be transformed into two seperate tokens, human_rights and human_biology, and therefore be more likely to be distinguished as different.

#### Training the Phraser

`train_phraser` has three stages. 
- First we create a list of tokenized *sentences*. 
- We then feed that list of sentences to a Gensim `Phrases` model. This model looks at which token co-occur, how often and [makes a judgement](https://arxiv.org/abs/1310.4546) about whether co-occurence is common enough to consider it a 'phrase'.
- Why sentences? Sentences mark out boundaries between words. Consider the phrase 'Human Rights'...


```
... and so recognising that he was only human. Rights based discussions can only....

```

```
... and so recognising that he was only human rights based discussions can only..

```

In [ ]:
# gensim is a text processing library that has the Phrasing tools we need
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

In [ ]:
def process_text(doc):
    return [token.lemma_.lower() for token in doc if token.text != '\n' and token.is_alpha]

# process sentences function
def process_sentences(doc):
    return [process_text(sent) for sent in doc.sents]

def train_phraser(texts, n_process=1):
    sentences = []
    for doc in nlp.pipe(texts, n_process=n_process):
        doc_sents = process_sentences(doc)
        sentences.extend(doc_sents)
    phrasing_model = Phrases(sentences, connector_words=ENGLISH_CONNECTOR_WORDS)
    return phrasing_model

In [ ]:
phraser = train_phraser(df['text'])

In [ ]:
test_text = nlp(df['text'].iloc[0])
test_text[:100]

In [ ]:
test_tokens = process_text(test_text)
print(test_tokens[10:35])

In [ ]:
test_tokens_phrased = phraser[test_tokens]
print(test_tokens_phrased[10:35])

In [ ]:
print([token for token in test_tokens_phrased if '_' in token])

In [ ]:
# We can save our phraser to disk so we don't have to do it again

phraser.save('phraser.bin')

# and load it when we need it

phraser = Phrases.load('phraser.bin')

In [ ]:
# OUR REPLACEMENT FUNCTIONS

def new_process_text(doc, phrasing_model=None):
    if phrasing_model is None:
        tokens = [token.lemma_.lower() for token in doc if token.text != '\n' and token.is_alpha]
    else:
        tokens = []
        sentences = process_sentences(doc)
        for sent in sentences:
            phrased = phrasing_model[sent]
            tokens.extend(phrased)
    return tokens 
    
def filter_stops(tokens, stop_words):
    return [tok for tok in tokens if tok.lower() not in stop_words]

def process_documents(texts, phraser_model=None, stop_words=None, n_process=1): #change here
    docs = nlp.pipe(texts, n_process=n_process)
    processed = [new_process_text(doc, phrasing_model=phraser_model) for doc in docs] # and here
    if stop_words is not None:
        processed = [filter_stops(doc, stop_words) for doc in processed]
    return processed

def stringify_tokens(tokenized_documents):
    return [' '.join(doc) for doc in tokenized_documents]


In [ ]:
print(new_process_text(test_text)[10:30])

In [ ]:
print(new_process_text(test_text, phrasing_model=phraser)[10:30])

In [ ]:
spacy_stop_words = nlp.Defaults.stop_words

tokenized_docs = process_documents(df['text'], phraser_model=phraser, stop_words=spacy_stop_words)

In [ ]:
print(tokenized_docs[3])

In [ ]:
# to reiterate the steps
df = pd.read_csv('sample_news_large.csv')

# Get your list of texts and stop words
corpus = df['text']
spacy_stop_words = nlp.Defaults.stop_words

# train your phraser and save it to avoid retraining later
phraser = train_phraser(corpus)
phraser.save('phraser.bin')

phraser = Phrases.load('phraser.bin')

tokenized_docs = process_documents(corpus, phraser_model=phraser, stop_words=spacy_stop_words)
# Done!!

In [ ]:
print(tokenized_docs[0])

In [ ]:
stringified = stringify_tokens(tokenized_docs)
print(stringified[0])

In [ ]:
df['tokens'] = stringified
df.head()

### Top Phrases
A useful exploratory technique to get a quick sense of your data is to examine the phrases used.

In [ ]:
df.head()

In [ ]:
def extract_phrases(tokens):
    return [token for token in tokens.split() if '_' in token]

df['phrases'] = df['tokens'].apply(extract_phrases)

In [ ]:
# phrases for one document
df['phrases'].iloc[0]

In [ ]:
# top ten phrases overall
print(df.explode('phrases')['phrases'].value_counts()[:10])

In [ ]:
# top ten phrases per group
for query,data in df.groupby('query'):
    print(f"****{query}****")
    print(data.explode('phrases')['phrases'].value_counts()[:10])
    print()